In [1]:
import os
import json
from PIL import Image

# Paths
coco_images_dir = "datasets/coco/images/val2017"
coco_annotations_file = "datasets/coco/annotations/instances_val2017.json"
output_dir = "datasets/coco_resized/images"

os.makedirs(output_dir, exist_ok=True)

In [2]:
# Load annotations
with open(coco_annotations_file, 'r') as f:
    coco_annotations = json.load(f)

In [3]:
# Function to resize image and update bounding boxes
def resize_image_and_boxes(image, annotations, target_divisor=64):
    w, h = image.size
    new_w = (w // target_divisor) * target_divisor
    new_h = (h // target_divisor) * target_divisor

    resized_image = image.resize((new_w, new_h))

    scale_x = new_w / w
    scale_y = new_h / h

    for ann in annotations:
        bbox = ann['bbox']
        ann['bbox'] = [
            bbox[0] * scale_x,
            bbox[1] * scale_y,
            bbox[2] * scale_x,
            bbox[3] * scale_y
        ]

    return resized_image, annotations

updated_annotations_list = []
# Process all images
for image_info in coco_annotations['images']:
    image_path = os.path.join(coco_images_dir, image_info['file_name'])
    image = Image.open(image_path)

    # Get annotations for the image
    image_annotations = [ann for ann in coco_annotations['annotations'] if ann['image_id'] == image_info['id']]

    # Resize image and update bounding boxes
    resized_image, updated_annotations = resize_image_and_boxes(image, image_annotations)

    # Save resized image
    output_image_path = os.path.join(output_dir, image_info['file_name'])
    resized_image.save(output_image_path)

    updated_annotations_list.extend(updated_annotations)

In [5]:
# Save updated annotations
resized_annotations_file = "datasets/coco_compressed/annotations/instances_new_val2017.json"
with open(resized_annotations_file, 'w') as f:
    json.dump(updated_annotations_list, f)